# Decision Trees

Decision trees are regarded as some of the most interpretable models, in that people with little training can make sense of them and put them to use to make decisions. The following decision tree can be used to estimate the annual income of someone.

![][1]

### Terms

A decision tree is composed of **nodes**, **branches**, and **leaves**. The nodes are represented by the rectangles in the diagram. At each node, a decision is made. In scikit-learn, all decision trees are composed of **binary** decisions. The condition evaluated at the node is either true or false producing two branches at each node. In general, decision trees can have any number of branches stemming from each node.

A branch is simply the path that one takes down the decision tree to reach the next node. All paths in a decision tree end at a leaf. The leaves contain the predicted value. In this example, the leaves hold an estimated annual salary.

### Regression vs Classification Decision trees

Decision trees can be used for both regression and classification learning problems. Regression problems have leaves that contain some numeric prediction. Classification problems have leaves with discrete classes. This chapter only focuses on regression decision trees. Together, these are formally referred to as **Classification and Regression Trees** or **CART**.

## How a decision tree is created

The creation of a decision tree is a computationally expensive process that involves a series of repeated steps. A summary of these steps is listed below.

1. At each node, create every single possible binary split of the data
1. Calculate the standard deviation of the target variable for each of the two groups
1. Calculate the weighted sum of these two standard deviations where the weight is proportional to the size of the group
1. Choose the binary split with minimum weighted standard deviation
1. Use this split to create two new nodes
1. Repeat the above steps for each node until some stopping criterion is met

[1]: images/tree.png

### Manually build a short tree

We'll now manually build a short decision tree to help better understand the steps it takes to create them. Let's begin by reading in the data selecting just `BedroomAbvGr` and `FullBath` as our features. The reason we use these two features is that they each contain a small number of unique values making the total number of binary splits much smaller than if we used other features like `GrLivArea` which contains hundreds of unique values.

In [ ]:
import pandas as pd
housing = pd.read_csv('../data/housing_sample.csv')
df = housing[['BedroomAbvGr', 'FullBath', 'SalePrice']]
df.head()

### Step 1 - Create every single possible binary split

At each node, we need to create every single possible binary split of the data. Let's begin with a concrete example of a single binary split of the data. We can divide the data into houses that have 2 or less bedrooms and those that have 3 or more.

In [ ]:
df_two_or_less_br = df.query('BedroomAbvGr <= 2')
df_three_or_more_br = df.query('BedroomAbvGr >= 3')

We have now split our data into two distinct DataFrames. Let's verify this by outputting the shape of each one.

In [ ]:
df_two_or_less_br.shape

In [ ]:
df_three_or_more_br.shape

We can also verify that the number of bedrooms in each DataFrame corresponds to how we split the data by counting the frequency of each unique value.

In [ ]:
df_two_or_less_br['BedroomAbvGr'].value_counts()

In [ ]:
df_three_or_more_br['BedroomAbvGr'].value_counts()

This is one example of a binary split of the data. We will be attempting every single possible binary split. To do this, we need to know the unique values of each feature. The function below, returns a dictionary mapping the feature name to the unique values. The Series `unique` method returns a numpy array which we sort it in-place with the `sort` method.

In [ ]:
def get_unique_values(df):
    unique_values = {}
    features = df.columns[:-1]
    for feature in features:
        unique = df[feature].unique()
        unique.sort()
        unique_values[feature] = unique
    return unique_values

The above function assumes that the target variable is the last column in the DataFrame. Let's verify that the function returns the sorted unique values.

In [ ]:
get_unique_values(df)

### List all the binary splits

Because there are relatively few unique values, we can list them out in their entirety. 

* 0 or less bedrooms <-> 1 or more
* 1 or less bedrooms <-> 2 or more
* 2 or less bedrooms <-> 3 or more
* 3 or less bedrooms <-> 4 or more
* 4 or less bedrooms <-> 5 or more
* 5 or less bedrooms <-> 6 or more
* 6 or less bedrooms <-> 7 or more
* 0 or less bathrooms <-> 1 or more
* 1 or less bathrooms <-> 2 or more
* 2 or less bathrooms <-> 3 or more

For each feature, there will be one less possible split than the number of unique values. For instance, there are eight unique values for bedrooms and seven possible binary splits. A total of 10 possible binary splits exist for the two features. Below, a function is defined to split the data given the feature name and the split value. It returns two DataFrames.

In [ ]:
def split(df, feature, val):
    df_left = df.query(f'{feature} <= {val}')
    df_right = df.query(f'{feature} > {val}')
    return df_left, df_right

Let's use this function to verify that it produces the same DataFrame shapes for a split value of 2 for bedrooms.

In [ ]:
df_left, df_right = split(df, 'BedroomAbvGr', 2)
df_left.shape

In [ ]:
df_right.shape

### Step 2 - Calculate the standard deviation of the target variable for each of the two groups

After making the split, you'll have two DataFrames. We now calculate the standard deviation of the target variable for each DataFrame.

In [ ]:
def std_groups(df_left, df_right):
    return df_left['SalePrice'].std(ddof=0), df_right['SalePrice'].std(ddof=0)

In [ ]:
std_groups(df_left, df_right)

### Step 3 - Calculate the weighted sum of these two standard deviations

You might be wondering why we calculated the standard deviation (square root of the variance). Ideally, we want each group to have as little variance as possible. The standard deviation is our objective measure of performance for rating the quality of the split. The lower the standard deviation, the better the split.

We have two standard deviations, but need a single number as our scoring metric. Each standard deviation is weighted in proportion to the number of observations of the DataFrame that it was calculated from. The sum of these weighted standard deviations is reported as the scoring metric for that particular split. The function below returns this metric. 

In [ ]:
def weighted_std(df_left, df_right):
    std_left, std_right = std_groups(df_left, df_right)
    num_rows_left = len(df_left)
    num_rows_right = len(df_right)
    N = num_rows_left + num_rows_right
    return std_left * num_rows_left / N + std_right * num_rows_right / N

There are more observations in `df_right` (1,046) than `df_left` (461), so the weighted standard deviation should be close to the standard deviation of `df_right`.

In [ ]:
weighted_std(df_left, df_right)

### Step 4 - Choose the binary split with minimum weighted standard deviation

We just calculated the weighted standard deviation for one split. We need to calculate this score for all of the possible splits. The function below returns a DataFrame of the scores of all the possible binary splits.

In [ ]:
def calculate_all_split_scores(df):
    unique_values = get_unique_values(df)
    scores = {'feature': [], 'split value': [], 'weighted std': []}
    for feature, values in unique_values.items():
        for val in values[:-1]:
            df_left, df_right = split(df, feature, val)
            score = weighted_std(df_left, df_right).round(0)
            scores['feature'].append(feature)
            scores['split value'].append(val)
            scores['weighted std'].append(score)
    return pd.DataFrame(scores)

We execute the function and return the scores for all splits.

In [ ]:
scores = calculate_all_split_scores(df)
scores

Now, we need to find the split with the minimum weighted standard deviation. The following function returns takes the above DataFrame of scores and returns the feature name and split value for the minimum score.

In [ ]:
def choose_best_split(scores):
    best_split = scores.nsmallest(1, 'weighted std')
    best_feature = best_split['feature'].values[0]
    best_split_value = best_split['split value'].values[0]
    return best_feature, best_split_value

Executing the function informs us that splitting the houses into those that have one or fewer bathrooms and two or more minimizes the standard deviation.

In [ ]:
choose_best_split(scores)

### Step 5 - Use this split to create two new nodes

Let's use our previously created `split` function to make our two new nodes.

In [ ]:
best_feature, best_split_value = choose_best_split(scores)
df_left, df_right = split(df, best_feature, best_split_value)

Output the shape to view how many observations are in each new node.

In [ ]:
df_left.shape

In [ ]:
df_right.shape

### Estimate Sale Price

We now have a very simple decision tree and can actually use it to estimate the sale price. Decision trees use the mean value of the observations in the leaves as the predicted value. Let's calculate the mean sale price of each group.

In [ ]:
df_left['SalePrice'].mean()

In [ ]:
df_right['SalePrice'].mean()

### View current state of tree

We can visualize this simple tree with the following diagram:

![][1]

[1]: images/stump.png

## Step 6 - Repeat the above steps for each node until some stopping criterion is met

The above five steps complete the process of finding the best binary split of the data with two new nodes produced. These same steps are repeated for each new node until a stopping criteria is met. A common stopping criterion is the **depth** of the tree. Currently, our tree has a depth of 1. There are many other stopping criterion, which will be discussed later when we use scikit-learn to build our tree.

The function `make_decision` below completes all the above steps, printing out the decision and returning the two new DataFrames. It also prints the mean target value for each new node.

In [ ]:
def make_decision(df):
    scores = calculate_all_split_scores(df)
    best_feature, best_split_value = choose_best_split(scores)
    df_left, df_right = split(df, best_feature, best_split_value)
    print(f'Decision: {best_feature} <= {best_split_value}')
    mean_left = df_left.iloc[:, -1].mean()
    print(f'mean left = {mean_left:,.0f}')
    mean_right = df_right.iloc[:, -1].mean()
    print(f'mean right = {mean_right:,.0f}')
    return df_left, df_right

Let's verify that this function produces the same result for our first decision.

In [ ]:
df_left, df_right = make_decision(df)

We can now use this function to continue splitting our nodes. Let's find out what the decision is for our left node and again for our right node.

In [ ]:
df_left_left, df_left_right = make_decision(df_left)

In [ ]:
df_right_left, df_right_right = make_decision(df_right)

While this function can be repeatedly called manually like this, we can improve upon it. The function `make_tree` below recursively builds the decision tree, storing the result as a dictionary. The tree stops growing whenever it reaches its maximum depth, which is set at two.

In [ ]:
def make_tree(df, tree=None, max_depth=2):
    if tree is None:
        tree = {'depth': 0}
    tree['mean'] = df.iloc[:, -1].mean()
    tree['N'] = len(df)
    if tree['depth'] < max_depth:
        scores = calculate_all_split_scores(df)
        best_feature, best_split_value = choose_best_split(scores)
        df_left, df_right = split(df, best_feature, best_split_value)
        tree['decision'] = f'{best_feature} <= {best_split_value}'
        tree['left'] = make_tree(df_left, {'depth': tree['depth'] + 1}, max_depth)
        tree['right'] = make_tree(df_right, {'depth': tree['depth'] + 1}, max_depth)
    return tree

Let's create the entire tree with one command and output the dictionary containing all of the tree info.

In [ ]:
tree = make_tree(df)
tree

The decision at each node is stored in the 'decision' key. Let's verify that the decision for the first split is the same.

In [ ]:
tree['decision']

The left and right nodes extending from each node are stored in the 'left' and 'right' keys. Let's select the left node.

In [ ]:
tree['left']

The depth, mean, N (number of observations), decision, and new nodes are output. Let's display just the decision for this node.

In [ ]:
tree['left']['decision']

We can access the left node extending from this current node. This is actually a leaf as it there are no other 'left' or 'right' keys in this dictionary.

In [ ]:
tree['left']['left']

The function `print_tree` below prints out the decision, number of observations, and mean at each node. It iterates through the tree using a breadth-first (horizontal) search as opposed to depth-first (vertical) which was how the tree was built.

In [ ]:
def print_tree(queue, tree_string=None, count=0):
    cur_tree = queue[0]
    if 'decision' in cur_tree:
        info = cur_tree['decision']
    else:
        info = f"leaf {cur_tree['mean']:,.0f}"
    depth = cur_tree['depth']
    decision = cur_tree.get('decision', 'leaf')
    N = cur_tree['N']
    mean = cur_tree['mean']
    space = {0: 0, 1: 20, 2:2}[depth] * ' '
    if count == 0:
        tree_string = ['', '', '']
        space = ''
    tree_string[0] += space + f'{decision:^15}'
    tree_string[1] += space + f'{f"N = {N}":^15}'
    tree_string[2] += space + f'{f"mean = {mean:,.0f}":^15}'
    count += 1
    if count == 2 ** depth:
        count = 0
        tree_string = [f'{string:^100}' for string in tree_string]
        print('\n'.join(tree_string))
        if decision != 'leaf':
            for i in range(10, 16, 2):
                s = f'/ {" ":{i}}  \\'
                s = [s] * (2 ** depth)
                s = space[:30 - i].join(s)
                print(f'{s:^100}')
    if 'left' in cur_tree:
        queue.append(cur_tree['left'])
        queue.append(cur_tree['right'])
    if len(queue) > 1:
        print_tree(queue[1:], tree_string, count)

Pass the function a one-item list of our tree.

In [ ]:
print_tree([tree])

A clearer visual of the just the decisions and predictions is shown in the following diagram.

![][1]

[1]: images/full_tree.png

## Use scikit-learn to create a decision tree

scikit-learn has its decision tree models available in the `tree` module. Let's first get our data assigned to the variables `X` and `y`.

In [ ]:
cols = ['BedroomAbvGr', 'FullBath']
X = housing[cols]
y = housing['SalePrice']

Now, let's import the `DecisionTreeRegressor` (and NOT the `DecisionTreeClassifier`). When we instantiate it, we are given several choices to control how the tree is built. Once of these choices is `max_depth` which controls the maximum depth of tree. We set this to two upon instantiation. Finally, we build the decision tree with a call to the `fit` method.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor(max_depth=2)
dtr.fit(X, y)

In order to visualize this tree, scikit-learn provides the `plot_tree` helper function. Below, we plot the tree and verify that our decision tree created above matches the one found with scikit-learn.

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
%matplotlib inline
fig, ax = plt.subplots(figsize=(16, 6))
plot_tree(dtr, fontsize=15, feature_names=cols, rounded=True, precision=1, filled=True, ax=ax);

### Make a prediction

Our decision tree above has four possible outcomes. If we have a one bedroom, one bathroom house, our model predicts 168,000. Let's see if the scikit-learn model agrees.

In [ ]:
dtr.predict([[1, 1]])

### Evaluate the model

Like all regression models, $R^2$ is returned from the `score` method.

In [ ]:
dtr.score(X, y)

### Inspect the tree

A tree object is available after training and is accessible as the `tree_` attribute.

In [ ]:
tree = dtr.tree_

The total number of nodes is found in the `node_count` attribute.

In [ ]:
tree.node_count

The number of leaves is found in the `n_leaves` attribute.

In [ ]:
tree.n_leaves

### Alternate visualization with graphviz

The `plot_tree` function from above renders the image using matplotlib. You can produce a sharper image with the help of the graphviz third-party library. You will need to install it before proceeding by executing the following command in your Anaconda Prompt/Terminal program. 

 `conda install python-graphviz`
 
You will probably need to install the [software GraphViz][1] as well and then run the following commands.

[1]: https://www.graphviz.org/download/

In [ ]:
import graphviz
from sklearn.tree import export_graphviz
dot_data = export_graphviz(dtr, feature_names=cols, precision=1, filled=True, rounded=True)  
graphviz.Source(dot_data, format='png')  

## Exercises

### Exercise 1

<span  style="color:green; font-size:16px">Make more decision trees using a different combination of features and depths. Be careful when visualizing very deep trees as there can be many thousands of nodes.</span>